In [1]:
import os.path

import matplotlib.pyplot as plt

plt.rcParams['text.usetex'] = True

import scanpy as sc
import pandas as pd
import seaborn as sns
# from outer_spacem.io import convert_name

import numpy as np
from pathlib import Path


# from outer_spacem.pl import plot_distributions, plot_umap_top_n, volcano_plot
from outer_spacem.pl._diff_expr import plot_distributions
# from singlecelltools.various import get_molecules_names

SAVE_ADATA = False
MAGIC = False

well = "Drug_W8"
export_only_significant = True
#
# well = "Feeding_W3"
# export_only_significant = False

analysis_name = "v4_check"
if MAGIC: analysis_name += "_magic"

main_dir = os.path.join("/Users/alberto-mac/EMBL_ATeam/projects/gastrosome",
                       well, "reprocessing")

# adata = sc.read(os.path.join(main_dir, "single_cell_analysis/spatiomolecular_adata.h5ad"))
# adata = sc.read("/Users/alberto-mac/Documents/DA_ESPORTARE/LOCAL_EMBL_FILES/scratch/projects/gastrosome_processing/SpaceM_processing_new/Drug_W8/analysis/single_cell_analysis/spatiomolecular_adata.h5ad")

# # //////////////////
# # New import where I import another well with fixed molecule IDs:
# metadata = pd.read_csv("/Users/alberto-mac/Documents/DA_ESPORTARE/LOCAL_EMBL_FILES/scratch/projects/gastrosome_processing_full/spacem/spacem_datasets_paths_final.csv")
# # metadata_subset = metadata.loc[metadata.index.isin([10]), :]
# # metadata_subset = metadata.loc[metadata.index.isin([3, 10]), :]
# metadata_subset = metadata
# # metadata_subset = metadata.loc[metadata.index.isin([0, 3]), :]
# import warnings
# import outer_spacem as osm
# pattern = "/Users/alberto-mac/Documents/DA_ESPORTARE/LOCAL_EMBL_FILES/scratch/projects/gastrosome_processing_full/spacem_synced/slide{slide}/W{well}/analysis/single_cell_analysis/spatiomolecular_adata.h5ad"
#
# warnings.simplefilter("ignore")
# adata = osm.io.bulk_read(
#     metadata_subset,
#     file_pattern = pattern
# )
# # Keep only drug treated cells:
# adata = adata[adata.obs.condition == "Drugs"]
# # mol_ids_col = "moleculeIds-0"
# # mol_names_col = "moleculeNames-0"

# //////////////////
# Old direct import:
adata = sc.read("/Users/alberto-mac/Documents/DA_ESPORTARE/LOCAL_EMBL_FILES/scratch/projects/gastrosome_processing_full/spacem_synced/slide6/W8/analysis/single_cell_analysis/spatiomolecular_adata.h5ad")
#
# # metadata_subset = metadata.loc[metadata.index.isin([3]), :]
# # adata2 = osm.io.bulk_read(
# #     metadata_subset,
# #     file_pattern = pattern
# # )
mol_ids_col = "moleculeIds"
mol_names_col = "moleculeNames"

# //////////////////


# intracell_ions = pd.read_csv("/Users/alberto-mac/EMBL_ATeam/projects/gastrosome/molecules_databases/reannotated/AB_Gastrosome_DrugW8_intra_ions_v2.tsv",
                             # sep="\t", index_col=0)

proj_dir = os.path.join(main_dir, "analysis", analysis_name)

plots_path = os.path.join(proj_dir, "plots")

os.makedirs(plots_path, exist_ok=True)
sc.settings.figdir = plots_path

In [2]:
# Get INCHI names:
from singlecelltools.various import get_inchi
MS_databases = [
    pd.read_csv(
        "/Users/alberto-mac/EMBL_ATeam/projects/gastrosome/molecules_databases/core_metabolome_v3.csv",
        sep="\t"),
    pd.read_csv(
        "/Users/alberto-mac/EMBL_ATeam/projects/gastrosome/molecules_databases/swisslipids_2018-02-02-v2.tsv",
        sep="\t")
]

In [3]:
adata.var

,annotation_id,formula,adduct,ionFormula,ion,mz,msm,fdr,databases,moleculeNames,moleculeIds
C5H12O7P2+H,C5H12O7P2+H,C5H12O7P2,+H,C5H13O7P2,C5H12O7P2+H+,247.013063,0.448323,NaN,"[[""Gastrosome_singlecell_intraions"", ""v1""]]","[""Dimethylallylpyrophosphate"", ""Isopentenyl py...","[""7526"", ""7527"", ""7528"", ""7529""]"
C5H12O7P2+K,C5H12O7P2+K,C5H12O7P2,+K,C5H12O7P2K,C5H12O7P2+K+,284.968945,0.000000,NaN,"[[""Gastrosome_singlecell_intraions"", ""v1""]]","[""Dimethylallylpyrophosphate"", ""Isopentenyl py...","[""7526"", ""7527"", ""7528"", ""7529""]"
C5H12O7P2+Na,C5H12O7P2+Na,C5H12O7P2,+Na,C5H12O7P2Na,C5H12O7P2+Na+,268.995008,0.000000,NaN,"[[""Gastrosome_singlecell_intraions"", ""v1""]]","[""Dimethylallylpyrophosphate"", ""Isopentenyl py...","[""7526"", ""7527"", ""7528"", ""7529""]"
C9H14N4O3+H,C9H14N4O3+H,C9H14N4O3,+H,C9H15N4O3,C9H14N4O3+H+,227.113828,0.000000,NaN,"[[""Gastrosome_singlecell_intraions"", ""v1""]]","[""Alanyl-Histidine"", ""Carnosine"", ""Histidinyl-...","[""7559"", ""7560"", ""7561""]"
C9H14N4O3+K,C9H14N4O3+K,C9H14N4O3,+K,C9H14N4O3K,C9H14N4O3+K+,265.069709,0.000000,NaN,"[[""Gastrosome_singlecell_intraions"", ""v1""]]","[""Alanyl-Histidine"", ""Carnosine"", ""Histidinyl-...","[""7559"", ""7560"", ""7561""]"
...,...,...,...,...,...,...,...,...,...,...,...
C57H101O7P+K,C57H101O7P+K,C57H101O7P,+K,C57H101O7PK,C57H101O7P+K+,967.691611,0.000000,NaN,"[[""Gastrosome_singlecell_intraions"", ""v1""]]","[""PA(O-18:1(11Z)/36:6(18Z,21Z,24Z,27Z,30Z,33Z)...","[""7493"", ""7494"", ""7495"", ""7496"", ""7497"", ""7498..."
C57H101O7P+Na,C57H101O7P+Na,C57H101O7P,+Na,C57H101O7PNa,C57H101O7P+Na+,951.717674,0.091886,NaN,"[[""Gastrosome_singlecell_intraions"", ""v1""]]","[""PA(O-18:1(11Z)/36:6(18Z,21Z,24Z,27Z,30Z,33Z)...","[""7493"", ""7494"", ""7495"", ""7496"", ""7497"", ""7498..."
C61H98O5+H,C61H98O5+H,C61H98O5,+H,C61H99O5,C61H98O5+H+,911.748663,0.000000,NaN,"[[""Gastrosome_singlecell_intraions"", ""v1""]]","[""TG(O-14:0/20:5(5Z,8Z,11Z,14Z,17Z)/24:6(6Z,9Z...","[""7530"", ""7531"", ""7532"", ""7533"", ""7534"", ""7535..."
C61H98O5+K,C61H98O5+K,C61H98O5,+K,C61H98O5K,C61H98O5+K+,949.704545,0.093179,NaN,"[[""Gastrosome_singlecell_intraions"", ""v1""]]","[""TG(O-14:0/20:5(5Z,8Z,11Z,14Z,17Z)/24:6(6Z,9Z...","[""7530"", ""7531"", ""7532"", ""7533"", ""7534"", ""7535..."


In [22]:
list_molecules_dbs = MS_databases
copy=True,
name_molecude_id_col="moleculeNames"
name_annotation_id_col="annotation_id"
name_out_inchi_col="inchi"
name_out_ids_col = 'correctMoleculeId'

new_df_data = {'annotation_id': [],
               'mol_index': [],
               'num_mols': [],
               'mol_id': []}

# Unroll lists in the adata.var dataframe:
for _, row in adata.var.iterrows():
    ids = eval(row[name_molecude_id_col])
    new_df_data["mol_id"] += ids
    new_df_data["mol_index"] += range(len(ids))
    new_df_data["annotation_id"] += [row[name_annotation_id_col] for _ in range(len(ids))]
    new_df_data["num_mols"] += [len(ids) for _ in range(len(ids))]
    # idx_range = range(len(ids))
new_df = pd.DataFrame(new_df_data)

# Now merge with METASPACE molecule databases:
combined_molecules_db = pd.concat(list_molecules_dbs)

In [6]:
# FIXME: this cell takes a while, make it more efficient, I could just remove double names
combined_molecules_wo_duplicates = []
for name, rows in combined_molecules_db.groupby("name"):
    selected = rows.iloc[0]
    if rows.shape[0] > 1:
        for i, id in enumerate(rows.id.sort_values(ascending=True)):
            if "HDB" in id:
                selected = rows.iloc[i]
                break
            elif "SLM" in id:
                selected = rows.iloc[i]
                break
    combined_molecules_wo_duplicates.append(selected)
combined_molecules_wo_duplicates = pd.DataFrame(combined_molecules_wo_duplicates)


In [28]:
combined_molecules_wo_duplicates.drop(columns="Unnamed: 0").reset_index(drop=True).to_csv("/Users/alberto-mac/EMBL_ATeam/projects/gastrosome/molecules_databases/merged_wo_duplicates.csv")

In [23]:
# Get rid of duplicates:

combined_molecules_wo_duplicates = pd.read_csv("/Users/alberto-mac/EMBL_ATeam/projects/gastrosome/molecules_databases/merged_wo_duplicates.csv")

merged_df = pd.merge(new_df, combined_molecules_wo_duplicates, how="left", left_on="mol_id", right_on="name")

# Finally, get inchi column in original format:
var_df = adata.var.copy()
inchi_lists = [None for _ in range(len(var_df))]
id_lists = [None for _ in range(len(var_df))]
for _, row in merged_df.iterrows():
    idx = var_df.index.get_loc(row[name_annotation_id_col])
    inchi = inchi_lists[idx]
    ids = id_lists[idx]
    if inchi is None:
        inchi = [None for _ in range(row.num_mols)]
        ids = [None for _ in range(row.num_mols)]
    inchi[row.mol_index] = str(row.inchi)
    ids[row.mol_index] = str(row.id)
    inchi_lists[idx] = inchi
    id_lists[idx] = ids

var_df[name_out_inchi_col] = inchi_lists
var_df[name_out_ids_col] = id_lists

# if copy:
#     return var_df
# else:
adata.var[name_out_inchi_col] = inchi_lists
adata.var[name_out_ids_col] = id_lists


